In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.cross_validation import StratifiedKFold
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from collections import Counter
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
from pandas.tools.plotting import parallel_coordinates
from sklearn.decomposition import PCA
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier

# So here are we with Forest Cover Analysis , that on various features we have to predict at least seven types of covers given to us ,
## Feature engg , polynomial features with pca and xgboost apart from lgbm we can get best result since being a handling various dataset, this problem is merely on few featured ensembel models
## Decision tree , base line model
## random forest , stackers,
## gbm , ada, xgboost and lgbm
## with taste of PCA too .
## evauation metrics will be accuracy , AUC , logloss and checking , variance and bias in final model¶

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
print('size of train data',train.shape)
print('size of test data',test.shape)

In [ ]:
train.head()

# As we can see that we are provided with non null dataset so we can more further by undertsanding outlier , feature scaling etc

In [ ]:
train.info()

In [ ]:
plt.figure(figsize=(12,5))
plt.title("Distribution of forest categories(Target Variable)")
ax = sns.distplot(train["Cover_Type"])

In [ ]:
sns.FacetGrid(train, hue="Cover_Type", size=10).map(plt.scatter, "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology").add_legend()

Parallel plots
A hillshade is a grayscale 3D representation of the surface, with the sun's relative position taken into account for shading the image. This function uses the altitude and azimuth properties to specify the sun's position.

Hillshade_9am (0 to 255 index) - Hillshade index at 9am, summer solstice
Hillshade_Noon (0 to 255 index) - Hillshade index at noon, summer solstice
Hillshade_3pm (0 to 255 index) - Hillshade index at 3pm, summer solstice


# From correlation analysis we can see that very few featues are depicting relation with one another a l like graph can been from top to bottom on LHS , VERY FEW soil type are showing some corrleation with another and other var as we can see some white lines has been drawn up this due tp heatmap scaling of points

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(train.corr(),fmt=".2f",cmap="YlGnBu")

In [ ]:
temp = train[['Hillshade_9am','Hillshade_Noon','Hillshade_3pm','Cover_Type']]
plt.figure(figsize=(15,11))
parallel_coordinates(temp,'Cover_Type', colormap=plt.get_cmap("Set1"))
plt.title("parallel plots of Hillshade with forest categories")
plt.xlabel("Hillshade")
plt.show()

In [ ]:
train.describe()

# AS we can see that mean vs max values lot of gap , outliers are here lot's of
## three types of elevation cover we can see with the help of quartiles
## now some interesting visualization by means of using correlation analysis
## as we can see slope is increasing and elevation to we are getting various forest covers so this variable is nececssary to know which type of cover

# Combining features for better visualization to get insights how cover is dependent pn soil and various other features

In [ ]:
soil_list = []
soil_not=[7,8,15,25]
for i in range(1, 41):
    if i not in soil_not:
       soil_list.append('Soil_Type' + str(i))

wilderness_area_list = ['Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4']

print(soil_list, "\n")
print(wilderness_area_list)

In [ ]:
pca = PCA(n_components=3) # for simplicity i choose 3
pca_results = pca.fit_transform(train.drop(["Cover_Type"], axis=1))
cmap = sns.cubehelix_palette(as_cmap=True)
tp, ax = plt.subplots(figsize=(20,15))
temp = ax.scatter(pca_results[:,0], pca_results[:,1], c=train.Cover_Type, s=50, cmap=cmap)
tp.colorbar(temp)
plt.show()

In [ ]:
def wilderness_compress(df):
    
    df[wilderness_area_list] = df[wilderness_area_list].multiply([1, 2, 3, 4], axis=1)
    df['Wilderness_Area'] = df[wilderness_area_list].sum(axis=1)
    return df

In [ ]:
def soil_compress(df):
    
    df[soil_list] = df[soil_list].multiply([i for i in range(1, 37)], axis=1)
    df['Soil_Type'] = df[soil_list].sum(axis=1)
    return df

In [ ]:
df = wilderness_compress(train)
df = soil_compress(train)

df[['Wilderness_Area', 'Soil_Type']].head()

In [ ]:
cols = df.columns.tolist()
columns = cols[1:11] + cols[56:]

print("Useful columns: ", columns)

values = df[columns]
labels = df['Cover_Type']

print("Values: ", values.shape)
print("Labels: ", labels.shape)

In [ ]:
clean = df[['Id', 'Cover_Type'] + columns]
clean.head()

Features engeneering :
Linear combination :
 * Elevation, Vertical dist. to Hydrology
 * Horizontal dist. to Hydrology, Horizontal dist. to Fire Points 
 * Horizontal dist. to Hydrology, Horizontal dist. to Roadways 
 * Horizontal dist. to Fire Points, Horizontal dist. to Roadways 
Euclidean distance :
  * Horizontal dist. to Hydrology, Vertical dist. to Hydrology
  * euclidean distance = \sqrt{(verticaldistance)^2 + (horizontaldistance)^2}\] 
Distance to Amenities :
As we know distances to amenities like Water, Fire and Roadways played a key role in determining cover type
Mean distance to Amenities
Mean Distance to Fire and Water

In [ ]:
####################### Train data #############################################
train['HF1'] = train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Fire_Points']
train['HF2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Fire_Points'])
train['HR1'] = abs(train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Roadways'])
train['HR2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Roadways'])
train['FR1'] = abs(train['Horizontal_Distance_To_Fire_Points']+train['Horizontal_Distance_To_Roadways'])
train['FR2'] = abs(train['Horizontal_Distance_To_Fire_Points']-train['Horizontal_Distance_To_Roadways'])
train['ele_vert'] = train.Elevation-train.Vertical_Distance_To_Hydrology

train['slope_hyd'] = (train['Horizontal_Distance_To_Hydrology']**2+train['Vertical_Distance_To_Hydrology']**2)**0.5
train.slope_hyd=train.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
train['Mean_Amenities']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
train['Mean_Fire_Hyd']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology) / 2 
####################### Test data #############################################
test['HF1'] = test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Fire_Points']
test['HF2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Fire_Points'])
test['HR1'] = abs(test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Roadways'])
test['HR2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Roadways'])
test['FR1'] = abs(test['Horizontal_Distance_To_Fire_Points']+test['Horizontal_Distance_To_Roadways'])
test['FR2'] = abs(test['Horizontal_Distance_To_Fire_Points']-test['Horizontal_Distance_To_Roadways'])
test['ele_vert'] = test.Elevation-test.Vertical_Distance_To_Hydrology

test['slope_hyd'] = (test['Horizontal_Distance_To_Hydrology']**2+test['Vertical_Distance_To_Hydrology']**2)**0.5
test.slope_hyd=test.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
test['Mean_Amenities']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
test['Mean_Fire_Hyd']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology) / 2


In [ ]:
train.columns

In [ ]:
import scipy.stats as stats
from scipy.stats import chi2_contingency

class ChiSquare:
    def __init__(self, dataframe):
        self.df = dataframe
        self.p = None #P-Value
        self.chi2 = None #Chi Test Statistic
        self.dof = None
        
        self.dfObserved = None
        self.dfExpected = None
        
    def _print_chisquare_result(self, colX, alpha):
        result = ""
        if self.p<alpha:
            result="{0} is IMPORTANT for Prediction".format(colX)
        else:
            result="{0} is NOT an important predictor. (Discard {0} from model)".format(colX)

        print(result)
        
    def TestIndependence(self,colX,colY, alpha=0.05):
        X = self.df[colX].astype(str)
        Y = self.df[colY].astype(str)
        self.dfObserved = pd.crosstab(Y,X) 
        chi2, p, dof, expected = stats.chi2_contingency(self.dfObserved.values)
        self.p = p
        self.chi2 = chi2
        self.dof = dof 
        
        self.dfExpected = pd.DataFrame(expected, columns=self.dfObserved.columns, index = self.dfObserved.index)
        
        self._print_chisquare_result(colX,alpha)

#Initialize ChiSquare Class
cT = ChiSquare(train)

#Feature Selection
testColumns = ['Id', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points',
       'Wilderness_Area1', 'Wilderness_Area2', 'Wilderness_Area3',
       'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3',
       'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7', 'Soil_Type8',
       'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12',
       'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16',
       'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20',
       'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24',
       'Soil_Type25', 'Soil_Type26', 'Soil_Type27', 'Soil_Type28',
       'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32',
       'Soil_Type33', 'Soil_Type34', 'Soil_Type35', 'Soil_Type36',
       'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40',
       'HF1', 'HF2', 'HR1', 'HR2', 'FR1', 'FR2', 'ele_vert',
       'slope_hyd', 'Mean_Amenities', 'Mean_Fire_Hyd']
for var in testColumns:
    cT.TestIndependence(colX=var,colY="Cover_Type" )  


In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df):
    # Get feature names
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]
    
    # Find interactions
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    # Remove interaction terms with all 0 values            
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df

In [ ]:
feature = [col for col in train.columns if col not in ['Cover_Type','Id']]
X_train = train[feature]
X_test = test[feature]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_train,train['Cover_Type'],test_size=.30)
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier
from sklearn.metrics import accuracy_score
rf=RandomForestClassifier(n_estimators=1100,max_depth=24,oob_score =True, random_state = 42,n_jobs=-1)
rf.fit(x_train,y_train)
y_pred=rf.predict(x_test)
print(accuracy_score(y_test,y_pred))
from sklearn.cross_validation import cross_val_score,cross_val_predict
score=cross_val_score(rf,x_train,y_train,n_jobs=-1,cv=5)
print(np.mean(score))

In [ ]:
plt.figure(figsize=(20,20))
predictors=list(x_train)
feat_imp = pd.Series(rf.feature_importances_, predictors).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Importance of Features')
plt.xlabel("feature")
plt.ylabel("feature importance")

In [ ]:
from sklearn.cross_validation import cross_val_score,cross_val_predict
y_pred=cross_val_predict(rf,x_test,y_test,n_jobs=-1,cv=5)
accuracy_score(y_test,y_pred)

In [ ]:
etc=ensemble.ExtraTreesClassifier(n_estimators=350,max_features="auto")
etc.fit(x_train,y_train)
y_pred=rf.predict(x_test)
accuracy_score(y_test,y_pred)
score=cross_val_score(etc,x_train,y_train,n_jobs=-1,cv=5)
print(np.mean(score))

#plt.scatter(range(300,2000,100),accuracy)
#plt.xlabel("no of trees")
#plt.ylabel("score")


In [ ]:
y_pred=cross_val_predict(etc,x_test,y_test,n_jobs=-1,cv=10)
accuracy_score(y_test,y_pred)

# the game changer move

In [ ]:
from sklearn.learning_curve import validation_curve
cv = StratifiedKFold(train['Cover_Type'], n_folds=10, shuffle=True, random_state=42)

# Before and after result of using startified kfold , i will be usnig startifield to predict the test data too , building the code and below it will be placed

In [ ]:
n_estimators_range = np.linspace(1, 200, 10).astype('int')
train_scores, test_scores = validation_curve(
    RandomForestClassifier(n_estimators=1100 ,max_depth=20,
 ),
    X_train, train['Cover_Type'],
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with Random Forest (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.7, 1.1)

plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

i = np.argmax(test_scores_mean)
print("Best cross-validation result ({0:.2f}) obtained for {1} trees".format(test_scores_mean[i], n_estimators_range[i]))


In [ ]:
n_estimators_range = np.linspace(1, 200, 10).astype('int')
train_scores, test_scores = validation_curve(
    RandomForestClassifier(n_estimators=1100 ,max_depth=20,
 ),
    X_train, train['Cover_Type'],
    param_name = 'n_estimators',
    param_range = n_estimators_range,
    cv=cv,
    n_jobs=-1,
    scoring='accuracy'
)

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with Random Forest (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.7, 1.1)

plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

i = np.argmax(test_scores_mean)
print("Best cross-validation result ({0:.2f}) obtained for {1} trees".format(test_scores_mean[i], n_estimators_range[i]))


# now we train and test data shuffled with stratified manner

In [ ]:
sub = pd.DataFrame({"Id": test['Id'],"Cover_Type": etc.predict(X_test)})
sub.to_csv("etcnew.csv", index=False) 

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
xgb=XGBClassifier(n_estimators=1100, max_depth=25)

In [ ]:
xgb.fit(x_train,y_train)
y_pred=xgb.predict(x_test)
print(accuracy_score(y_test,y_pred))
score=cross_val_score(xgb,x_train,y_train,n_jobs=-1,cv=5)
print(np.mean(score))

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

fig = plt.figure(figsize=(10, 6), dpi=100)

plt.title("Validation Curve with XGBoost (eta = 0.3)")
plt.xlabel("number of trees")
plt.ylabel("Accuracy")
plt.ylim(0.7, 1.1)

plt.plot(n_estimators_range,
             train_scores_mean,
             label="Training score",
             color="r")

plt.plot(n_estimators_range,
             test_scores_mean, 
             label="Cross-validation score",
             color="g")

plt.fill_between(n_estimators_range, 
                 train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, 
                 alpha=0.2, color="r")

plt.fill_between(n_estimators_range,
                 test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std,
                 alpha=0.2, color="g")

plt.axhline(y=1, color='k', ls='dashed')

plt.legend(loc="best")
plt.show()

i = np.argmax(test_scores_mean)
print("Best cross-validation result ({0:.2f}) obtained for {1} trees".format(test_scores_mean[i], n_estimators_range[i]))


# Plotting Roc Curves for each classes 

# stay tuned light gb , and more visualization are coming

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import np_utils

In [ ]:
X_train 
y_train = train.Cover_Type
y_train = np_utils.to_categorical(y_train)

In [ ]:
# Create model
model = Sequential()
model.add(Dense(1024, input_dim=64, kernel_initializer='uniform', activation='selu'))
model.add(Dense(512, kernel_initializer='uniform', activation='softplus'))
model.add(Dense(256, kernel_initializer='uniform', activation='elu'))
model.add(Dense(128, kernel_initializer='uniform', activation='selu'))
model.add(Dense(64, kernel_initializer='uniform', activation='softplus'))
model.add(Dense(32, kernel_initializer='uniform', activation='elu'))
model.add(Dense(16, kernel_initializer='uniform', activation='softplus'))
model.add(Dense(8, kernel_initializer='uniform', activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit model
model.fit(X_train, y_train, epochs=10000, batch_size=32)